# Strides

In [68]:
import numpy as np

X = np.arange(12, dtype=np.int64).reshape(3, 4)

print("C-contiguous:")
print(X.shape)
print(X.itemsize)
print(X.strides)
print(X.flags['C_CONTIGUOUS'])

print("\n")

Y = np.asfortranarray(X)
print("Fortran-contiguous:")
print(Y.shape)
print(Y.itemsize)
print(Y.strides)
print(Y.flags['C_CONTIGUOUS'])



C-contiguous:
(3, 4)
8
(32, 8)
True


Fortran-contiguous:
(3, 4)
8
(8, 24)
False


### Transpose

In [70]:
Z = X.T

print("Z.shape:", Z.shape)
print("Z.strides:", Z.strides)
print("Z.base is X:", Z.base is X.base)


Z.shape: (4, 3)
Z.strides: (8, 32)
Z.base is X: True


### as_strided

In [55]:
def sliding_window_view_1d(x, w):
    # x: 1D, w: window size
    x = np.ascontiguousarray(x)
    n = x.shape[0]
    shape   = (n - w + 1, w)
    strides = (x.strides[0], x.strides[0])
    return np.lib.stride_tricks.as_strided(x, shape=shape, strides=strides)

x = np.arange(10, dtype=np.int8)
W = sliding_window_view_1d(x, 4)



print("x:\n", x)
print("x.dtype:", x.dtype)
print("x.strides:", x.strides)
print("x.shape:", x.shape)

print("\n")
      
print("W:\n", W)
print("W.strides:", W.strides)


x:
 [0 1 2 3 4 5 6 7 8 9]
x.dtype: int8
x.strides: (1,)
x.shape: (10,)


W:
 [[0 1 2 3]
 [1 2 3 4]
 [2 3 4 5]
 [3 4 5 6]
 [4 5 6 7]
 [5 6 7 8]
 [6 7 8 9]]
W.strides: (1, 1)


`as_strided` doesn’t check bounds — if your new shape+strides reach outside the original buffer,
you’ll read garbage or segfault.
Hence, we often guard it with `np.lib.stride_tricks.as_strided`


In [72]:
def sliding_window_view_2d(X, wh, ww):
    X = np.ascontiguousarray(X)
    H, W = X.shape
    oh, ow = H - wh + 1, W - ww + 1
    sh, sw = X.strides
    shape   = (oh, ow, wh, ww)
    strides = (sh, sw, sh, sw)
    return np.lib.stride_tricks.as_strided(X, shape=shape, strides=strides)

X = np.arange(5*6).reshape(5,6)

P = sliding_window_view_2d(X, 3, 3)

print("X:\n", X)
print("\nX.strides:", X.strides)

print("\nP.shape:", P.shape)
print("\nP.strides:", P.strides)

print("\nP:\n", P)

print("\n\nAverage of each 3x3 window:")
print(P.mean(axis=(2,3)))



X:
 [[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]]

X.strides: (48, 8)

P.shape: (3, 4, 3, 3)

P.strides: (48, 8, 48, 8)

P:
 [[[[ 0  1  2]
   [ 6  7  8]
   [12 13 14]]

  [[ 1  2  3]
   [ 7  8  9]
   [13 14 15]]

  [[ 2  3  4]
   [ 8  9 10]
   [14 15 16]]

  [[ 3  4  5]
   [ 9 10 11]
   [15 16 17]]]


 [[[ 6  7  8]
   [12 13 14]
   [18 19 20]]

  [[ 7  8  9]
   [13 14 15]
   [19 20 21]]

  [[ 8  9 10]
   [14 15 16]
   [20 21 22]]

  [[ 9 10 11]
   [15 16 17]
   [21 22 23]]]


 [[[12 13 14]
   [18 19 20]
   [24 25 26]]

  [[13 14 15]
   [19 20 21]
   [25 26 27]]

  [[14 15 16]
   [20 21 22]
   [26 27 28]]

  [[15 16 17]
   [21 22 23]
   [27 28 29]]]]


Average of each 3x3 window:
[[ 7.  8.  9. 10.]
 [13. 14. 15. 16.]
 [19. 20. 21. 22.]]


### Tilings

In [61]:
x = np.array([1,2,3])
shape   = (4, 3)
strides = (0, x.strides[0])
tile = np.lib.stride_tricks.as_strided(x, shape=shape, strides=strides)

print("tile:\n", tile)
print("\ntile.strides:", tile.strides)

tile:
 [[1 2 3]
 [1 2 3]
 [1 2 3]
 [1 2 3]]

tile.strides: (0, 8)


## Example: Hankel matrix

In [63]:
x = np.arange(1, 11)
N = x.size
L = 5

# ---- Hankel matrix (each anti-diagonal constant) ----
# Shape: (rows, cols) = (L, N-L+1)
# Strides: row step = itemsize, col step = itemsize
# but columns start at successive positions; we stack L rows each shifted by +1
s = x.strides[0]
H = np.lib.stride_tricks.as_strided(x, shape=(L, N-L+1), strides=(s, s))

print("x:\n", x)
print("\nx.strides:", x.strides)

print("\nH:\n", H)
print("\nH.strides:", H.strides)

x:
 [ 1  2  3  4  5  6  7  8  9 10]

x.strides: (8,)

H:
 [[ 1  2  3  4  5  6]
 [ 2  3  4  5  6  7]
 [ 3  4  5  6  7  8]
 [ 4  5  6  7  8  9]
 [ 5  6  7  8  9 10]]

H.strides: (8, 8)


## Example: Toeplitz matrix

In [40]:
T = H[::-1, :]
print("T:\n", T)

T:
 [[ 5  6  7  8  9 10]
 [ 4  5  6  7  8  9]
 [ 3  4  5  6  7  8]
 [ 2  3  4  5  6  7]
 [ 1  2  3  4  5  6]]
